In [ ]:
!pip3 install opentsne==0.5.0

In [ ]:
import openTSNE
openTSNE.__version__

In [ ]:
from sklearn import metrics
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold

from sklearn.decomposition import PCA

from openTSNE import TSNE
from openTSNE.callbacks import ErrorLogger

import umap

import matplotlib.cm as cm
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

def show_grid(data):
    plt.figure(figsize=(16, 6))
    for i in range(10):
        plt.subplot(2, 5, i + 1)
        plt.imshow(data[i,:].reshape([28,28]), cmap='gray');

def plot_2d_components(data_x,data_y,title):
    plt.figure(figsize=(12,10))
    plt.scatter(data_x[:, 0], data_x[:, 1], c=list(map(int,data_y)), 
                edgecolor='none', alpha=0.7, s=40,
                cmap=plt.cm.get_cmap('nipy_spectral', 10))
    plt.colorbar()
    plt.title(title)
    
def estimate_clf_score(clf,X_train,X_test,y_train,y_test):
    cv = StratifiedKFold(n_splits=5, random_state=123, shuffle=True)
    accuracy_s = []

    for (train_index, test_index), i in zip(cv.split(X_train, y_train), range(5)):
        clf.fit(X_train[train_index], y_train[train_index])
        accuracy = clf.score(X_train[test_index], y_train[test_index])
        accuracy_s.append(accuracy * 100)
    
    clf.fit(X_train, y_train)
    
    print("Train mean accuracy: %.2f%% (std=%.2f)" % (np.mean(accuracy_s),np.std(accuracy_s)))
    print("Test mean accuracy:  %.2f%%" % (clf.score(X_test, y_test) * 100))

    

# Загрузим данные MNIST

In [ ]:
# Load data from https://www.openml.org/d/554
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
X.shape

In [ ]:
test_size = 10000
X_train, X_test = X[:test_size], X[test_size:]
y_train, y_test = y[:test_size], y[test_size:]
print("Test size is %.2f%% of dataset" % (test_size * 100 / X.shape[0]))
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Применим слабый классификатор

In [ ]:
%%time
clf = RandomForestClassifier(max_depth=3, n_estimators=100)
estimate_clf_score(clf,X_train,X_test,y_train,y_test)

# PCA

## Визуализация через PCA

In [ ]:
components=2
print('Projecting %d-dimensional data to %dD' % (X_train.shape[1],components))
pca = PCA(n_components=components)
X_train_reduced = pca.fit_transform(X_train)
X_test_reduced = pca.transform(X_test)

In [ ]:
plot_2d_components(X_train_reduced,y_train,'MNIST. PCA projection')

## Кривая обьясняемой дисперсии

In [ ]:
pca = PCA()
X_train_reduced = pca.fit_transform(X_train)
X_test_reduced = pca.transform(X_test)

In [ ]:
plt.figure(figsize=(10,7))
ratio_sums = np.cumsum(pca.explained_variance_ratio_)
plt.plot(ratio_sums, color='k', lw=2)
plt.xlabel('Number of components')
plt.ylabel('Total explained variance')
plt.xlim(0, 748)
plt.yticks(np.arange(0, 1.1, 0.1))
enough_components = np.where(ratio_sums >= 0.9)[0][0]
plt.axvline(enough_components, c='b')
plt.axhline(0.9, c='r')
plt.show();

## Попробуем помочь слабому классификатору

In [ ]:
%%time
components=enough_components
print('Projecting %d-dimensional data to %dD' % (X_train.shape[1],components))
pca = PCA(n_components=components)
X_train_reduced = pca.fit_transform(X_train)
X_test_reduced = pca.transform(X_test)

clf = RandomForestClassifier(max_depth=3, n_estimators=100)

estimate_clf_score(clf,X_train_reduced,X_test_reduced,y_train,y_test)

## Визуализация компонентов PCA

In [ ]:
show_grid(pca.components_)

# t-SNE

In [ ]:
%%time

tsne = TSNE(
    perplexity=50,
    metric="euclidean",
    verbose=True,
    n_jobs=4,
    random_state=1,
)

tsne_embedding_train = tsne.fit(X_train)

In [ ]:
%%time
X_test_tsne = tsne_embedding_train.transform(X_test)

In [ ]:
plot_2d_components(tsne_embedding_train,y_train,'MNIST. t-SNE projection')

## Попробуем применить к слабому классификатору

In [ ]:
%%time
clf = RandomForestClassifier(max_depth=3, n_estimators=100)
estimate_clf_score(clf,tsne_embedding_train,X_test_tsne,y_train,y_test)

## Можно немного ускориться PCA + t-SNE

In [ ]:
%%time

pca = PCA(n_components=20)
X_train_reduced = pca.fit_transform(X_train)
X_test_reduced = pca.transform(X_test)

tsne = TSNE(
    n_components=2,
    perplexity=50,
    metric="euclidean",
    verbose=True,
    n_jobs=4,
    random_state=1,
)

tsne_embedding_train_2 = tsne.fit(X_train_reduced)

In [ ]:
plot_2d_components(tsne_embedding_train_2,y_train,'MNIST. t-SNE projection')

# Параметры t-SNE

## n_components 

Количество компонентов или размерность конечного пространства. 

## perplexity

Количество локальных соседей каждой точки которео используется для каждой точки при поиске структуры многообразия.

### perplexity = 10

In [ ]:
tsne = TSNE(
    n_components=2,
    perplexity=10,
    metric="euclidean",
    verbose=False,
    n_jobs=4,
    random_state=1,
)

tsne_embedding_train_2 = tsne.fit(X_train_reduced)
plot_2d_components(tsne_embedding_train_2,y_train,'MNIST. t-SNE projection')

### perplexity = 100

In [ ]:
tsne = TSNE(
    n_components=2,
    perplexity=100,
    metric="euclidean",
    verbose=False,
    n_jobs=4,
    random_state=1,
)

tsne_embedding_train_2 = tsne.fit(X_train_reduced)
plot_2d_components(tsne_embedding_train_2,y_train,'MNIST. t-SNE projection')

## early_exaggeration
Управляет тем, насколько плотно естественные кластеры исходного пространства находятся во встроенном пространстве и сколько места будет между ними. Для больших значений пространство между кластерами будет больше в финальном пространстве.


По умолчанию 12

### early_exaggeration = 32

In [ ]:
tsne = TSNE(
    n_components=2,
    early_exaggeration=32,
    perplexity=50,
    metric="euclidean",
    verbose=False,
    n_jobs=4,
    random_state=1,
)

tsne_embedding_train_2 = tsne.fit(X_train_reduced)
plot_2d_components(tsne_embedding_train_2,y_train,'MNIST. t-SNE projection')

## learning_rate, n_iter

Значения по умолчанию:
- learning_rate=200.0 (обычно находится в промежутке [10.0, 1000.0])
- n_iter=1000

# UMAP

In [ ]:
%%time
umap_reducer = umap.UMAP(random_state=1)
umap_embedding = umap_reducer.fit_transform(X_test)

plot_2d_components(umap_embedding,y_test,'MNIST. UMAP projection')

## n_components

Количество компонентов или размерность конечного пространства. 

По умолчанию 2

## min_dist

Параметр min_dist контролирует, насколько тесно UMAP разрешено упаковывать точки вместе внутри низкоразмерного кластера. 

По умолчанию 0.1

### min_dist = 0.0

In [ ]:
%%time
umap_reducer = umap.UMAP(random_state=1,min_dist=0.0)
umap_embedding = umap_reducer.fit_transform(X_test)

plot_2d_components(umap_embedding,y_test,'MNIST. UMAP projection')

### min_dist = 0.3

In [ ]:
%%time
umap_reducer = umap.UMAP(random_state=1,min_dist=0.3)
umap_embedding = umap_reducer.fit_transform(X_test)

plot_2d_components(umap_embedding,y_test,'MNIST. UMAP projection')

### min_dist = 0.5

In [ ]:
%%time
umap_reducer = umap.UMAP(random_state=1,min_dist=0.5)
umap_embedding = umap_reducer.fit_transform(X_test)

plot_2d_components(umap_embedding,y_test,'MNIST. UMAP projection')

## n_neighbors

Этот параметр определяет, как UMAP балансирует между локальной и глобальной структурой данных. Это достигается за счет ограничения размера локального окружения, на которое UMAP будет обращать внимание при попытке изучить структуру данныз. 

Это означает, что низкие значения n_neighbors заставят UMAP сосредоточиться на очень локальной структуре (потенциально в ущерб общей картине), в то время как большие значения заставят UMAP смотреть на более крупные окрестности каждой точки при оценке структуры данных, потеря структуры мелких деталей ради получения более широких данных.

По умолчанию 15.

### n_neighbors = 5

In [ ]:
%%time
umap_reducer = umap.UMAP(random_state=1,n_neighbors=5)
umap_embedding = umap_reducer.fit_transform(X_test)

plot_2d_components(umap_embedding,y_test,'MNIST. UMAP projection')

### n_neighbors = 25

In [ ]:
%%time
umap_reducer = umap.UMAP(random_state=1,n_neighbors=25)
umap_embedding = umap_reducer.fit_transform(X_test)

plot_2d_components(umap_embedding,y_test,'MNIST. UMAP projection')

### n_neighbors = 100

In [ ]:
%%time
umap_reducer = umap.UMAP(random_state=1,n_neighbors=100)
umap_embedding = umap_reducer.fit_transform(X_test)

plot_2d_components(umap_embedding,y_test,'MNIST. UMAP projection')

# Интересные моменты

## Нелинейные методы и шум

In [ ]:
X = np.random.rand(1000,10)
X[:3,:5]

In [ ]:
tsne = TSNE(
    n_components=2,
    perplexity=5,
    metric="euclidean",
    verbose=False,
    n_jobs=4,
    random_state=1,
)

X_embedding = tsne.fit(X)


plt.figure(figsize=(12,10))
plt.scatter(X_embedding[:, 0], X_embedding[:, 1])

In [ ]:
# np.arange(len(X))
X[:,0] = np.arange(len(X))
X[:3,:5]

## Нелинейные методы и ошибки в данных

In [ ]:
tsne = TSNE(
    n_components=2,
    perplexity=5,
    metric="euclidean",
    verbose=False,
    n_jobs=4,
    random_state=1,
)

X_embedding = tsne.fit(X)


plt.figure(figsize=(12,10))
plt.scatter(X_embedding[:, 0], X_embedding[:, 1])

# Дополнительные материалы

- Туториал по использованию t-SNE https://distill.pub/2016/misread-tsne/
- Масса туториалов по UMAP https://umap-learn.readthedocs.io/en/latest/
- Красивая статья про UMAP и t-SNE https://pair-code.github.io/understanding-umap/